# BOEING CHALLENGE

#### by: Nicholas Matveev

In [34]:
import requests
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

# Connect to the whole dataset via REST API over at 
# https://data.gov.au/data/dataset/international-airlines-airline-by-country-of-port-data/resource/809c77d8-fd68-4a2c-806f-c63d64e69842
# using &limit=100000 parameter to pull the whole dataset:

out = requests.get('https://data.gov.au/data/api/3/action/datastore_search?resource_id=809c77d8-fd68-4a2c-806f-c63d64e69842&limit=100000')

#Converts the returned JSON into dataframe by looking at the table located at /result/records child node:
out_df = pd.json_normalize(out.json(),[['result','records']]) 

In [35]:
# 1.How many passengers came in and out of Australia for each Airline in each month of 2019?

#Pull the 2019 subset of the data:
df=out_df.copy().loc[out_df['Year']=='2019']

#Convert Passengers_In, Passengers_Out and Month_num column datatypes from string to numeric:
df[['Passengers_In','Passengers_Out','Month_num']]=df[['Passengers_In','Passengers_Out','Month_num']].apply(pd.to_numeric, errors='coerce')
df=df.sort_values(['Airline','Month_num'])

# Group by Airline and month number (1-12) to obtain the sum for passengers in and out for each month:
df.groupby(['Airline','Month_num'])[['Passengers_In','Passengers_Out']].sum().reset_index()

,Airline,Month_num,Passengers_In,Passengers_Out
0,Air Caledonie International,1,8291,7336
1,Air Caledonie International,2,3889,6148
2,Air Caledonie International,3,3957,4409
3,Air Caledonie International,4,4965,4911
4,Air Caledonie International,5,3996,3786
5,Air Caledonie International,6,4001,5143
6,Air Caledonie International,7,3969,4200
7,Air Caledonie International,8,5575,5017
8,Air Caledonie International,9,4377,4832
9,Air Caledonie International,10,5962,5901


In [36]:
# 2.In the last 6 months which port country was responsible for the most passengers coming into the country?

#Pull the 2020 subset of the data:
df=out_df.copy().loc[out_df['Year']=='2020']

#Convert month and Passengers__In to numeric datatype:
df[['Month','Passengers_In']]=df[['Month','Passengers_In']].apply(pd.to_numeric, errors='coerce')

#Get unique last 6 'Month' id values and store in last_six_months variable as a list and 
#keep rows that only refer to the last 6 months of the dataset (using 'Month' unique values):
last_six_months=df['Month'].drop_duplicates().nlargest(6).reset_index(drop=True).values.tolist()
df = df[df['Month'].isin(last_six_months)]

#Group by Port_Country to sum the Passengers coming into the country. Sort by descending and keep only the top row
df.groupby('Port_Country')[['Passengers_In']].sum().sort_values('Passengers_In',ascending=False).head(1)

,Passengers_In
Port_Country,
New Zealand,26803


In [37]:
# 3.What is the 3 month rolling average of freight coming into Australia for each airline in 2018?

#Pull the 2018 subset of the data:
df=out_df.copy().loc[out_df['Year']=='2018']
df[['Month_num','Freight_In_(tonnes)']]=df[['Month_num','Freight_In_(tonnes)']].apply(pd.to_numeric, errors='coerce')

#Take only 'Airline','Freight_In_(tonnes)','Month_num' columns from the dataset 
#and sort by the Airline and Month_Num ascending order to make it easier to read:
df = df.loc[:,['Airline','Freight_In_(tonnes)','Month_num']].sort_values(['Airline','Month_num'])

#Group by Airline and Month number to get the sum for each available Airline and Month combination:
df = df.groupby(['Airline','Month_num'])['Freight_In_(tonnes)'].sum().reset_index()

#Group by Airline and apply rolling 3 month mean for the summed Freight_In variable, for each Airline in 2018:
#Note: 3 month rolling average starts strictly on the 3rd month, but could change the rolling(3) function to 
#      rolling(3,1) to avoid missing values for the first couple of months for each airline

df['3_Month_MA'] = df.groupby(['Airline'])['Freight_In_(tonnes)'].rolling(3).mean().reset_index(drop=True)
df.round(3)

,Airline,Month_num,Freight_In_(tonnes),3_Month_MA
0,Air Caledonie International,1,2.942,NaN
1,Air Caledonie International,2,4.546,NaN
2,Air Caledonie International,3,4.184,3.891
3,Air Caledonie International,4,8.042,5.591
4,Air Caledonie International,5,2.332,4.853
5,Air Caledonie International,6,4.765,5.046
6,Air Caledonie International,7,2.681,3.259
7,Air Caledonie International,8,7.005,4.817
8,Air Caledonie International,9,4.509,4.732
9,Air Caledonie International,10,4.714,5.409


In [38]:
#4.Which 3 airlines have the highest mail inbound/outbound ratio across the whole dataset?

#Pull only the relevant Airline, Mail_In and Mail_Out columns:
df=out_df.copy().loc[:,['Airline','Mail_In_(tonnes)','Mail_Out_(tonnes)']]
df[['Mail_In_(tonnes)','Mail_Out_(tonnes)']]=df[['Mail_In_(tonnes)','Mail_Out_(tonnes)']].apply(pd.to_numeric, errors='coerce')

#Group by Airline to get the sum for both Mail in and out columns:
df = df.groupby(['Airline'])[['Mail_In_(tonnes)','Mail_Out_(tonnes)']].sum().reset_index()

#Filter Mail_out to avoid division by 0:
df = df.loc[df['Mail_Out_(tonnes)']!=0]

#Apply new column Mail_In/Out ratio to the dataframe:
df['Mail_In/Out']=df['Mail_In_(tonnes)']/df['Mail_Out_(tonnes)']

#Sort the ratio by descending and keep the top 3:
df.sort_values('Mail_In/Out',ascending=False).reset_index(drop=True).head(3)

,Airline,Mail_In_(tonnes),Mail_Out_(tonnes),Mail_In/Out
0,Asiana Airlines,5203.020,0.097,53639.381443
1,Middle East Airlines,17.460,0.012,1455.000000
2,Aeroflot Russian Intl A/l,6.922,0.012,576.833333


In [39]:
#5.BONUS: Using the dataset, identify an airline that may have changed name and explain your approach?

# The overall approach for the problem:

# 1. Group by Airline and date to get monthly stats for each Airline (use "sum" for numeric variables and "list" for Port_country variable)
# 2. In the dataset, get the date of first and last occurence of an Airline in the dataset
# 3. Find potential matches based on criteria: old airline last date occurrence = new airline first occurrence - 1 month
#    This assumes the airline that changed names kept operating without interruption and new airline name appeared in the dataset
#    1 month after the old airline name stopped appearing
# 4. Rank matches in terms of old airline and new airline feature similarity (e.g. flights, port countries, freight etc.)
# 5. Deduplicate matches and sort by combined feature score to obtain matches from best to worst


df=out_df.copy()

#Create a datetime column by combining Year and Month_Num columns (automatically takes "day" to be the 1st of each month):
df['datetime']=pd.to_datetime(df["Year"]+df["Month_num"],format='%Y%m')

#Convert all the relevant numeric columns to numeric type:
df[['Passengers_In','Passengers_Out','Freight_In_(tonnes)','Freight_Out_(tonnes)','Mail_In_(tonnes)','Mail_Out_(tonnes)']]=df[['Passengers_In','Passengers_Out','Freight_In_(tonnes)','Freight_Out_(tonnes)','Mail_In_(tonnes)','Mail_Out_(tonnes)']].apply(pd.to_numeric, errors='coerce')
df=df.sort_values(['datetime','Airline','Port_Country']).reset_index(drop=True)

#Group by Airline and datetime, to obtain aggregate for each month of each Airline:
#Note: Port_Country as a string will contain all Port_Country occurrences for each month, separated by a comma (useful to compare port countries of old and new airline name to find similarities later on)
df_monthly_stats = df.groupby(['Airline','datetime']).agg(
                                                Passengers_In=('Passengers_In','sum'),
                                                Passengers_Out=('Passengers_Out','sum'),
                                                Freight_In=('Freight_In_(tonnes)','sum'),
                                                Freight_Out=('Freight_Out_(tonnes)','sum'),
                                                Mail_In=('Mail_In_(tonnes)','sum'),
                                                Mail_Out=('Mail_Out_(tonnes)','sum'),
                                                Port_Country=('Port_Country',','.join)).reset_index()

#Convert Airline to upper case for more accurate matching of old and new Airline name strings:
df_monthly_stats['Airline'] = df_monthly_stats['Airline'].str.upper()

#Append to the monthly stats dataframe, the date of the first and last occurence of the Airline name throughout the whole dataset:
df_monthly_stats['first_occurrence']=df_monthly_stats.groupby(['Airline'])['datetime'].transform('first') 
df_monthly_stats['last_occurrence']=df_monthly_stats.groupby(['Airline'])['datetime'].transform('last') 

#Separate dataframes into first and last time airline occurrences for comparison
df_last_occurrences = df_monthly_stats.copy().loc[(df_monthly_stats['last_occurrence']==df_monthly_stats['datetime'])]  
df_first_occurrences = df_monthly_stats.copy().loc[(df_monthly_stats['first_occurrence']==df_monthly_stats['datetime'])]

#Set first occurrence as the month before to have a common join key (i.e. last occurence old airline name = first occurrence new airline name -1)
df_first_occurrences['first_occurrence'] = df_first_occurrences['first_occurrence']- pd.DateOffset(months=1)

In [40]:
#Using recordlinkage package to compare monthly stats features of airlines under the old and new company names:

import recordlinkage

#Set the index on which recordlinkage will link the data to "block". This means it will use exact match of last and first occurrence of airline names:
indexer = recordlinkage.Index()
indexer.block(left_on='last_occurrence', right_on='first_occurrence')
#Find potential candidates based on
candidates = indexer.index(df_last_occurrences, df_first_occurrences)

#For the compare class, compare each relevant feature between the linked airlines (including the airline name itself in case it's only a minor naming change)
compare = recordlinkage.Compare()
compare.string('Port_Country',
            'Port_Country',
            method='jarowinkler',
            threshold=0.75,
            label='Port_Country_Score')
compare.string('Airline',
            'Airline',
            method='jarowinkler',
            threshold=0.75,
            label='Airline_Score')
compare.numeric('Passengers_In',
               'Passengers_In',
                label='Passenger_In'
               )
compare.numeric('Passengers_Out',
               'Passengers_Out',
                label='Passenger_Out'
               )
compare.numeric('Freight_In',
               'Freight_In',
                label='Freight_In'
               )
compare.numeric('Freight_Out',
               'Freight_Out',
                label='Freight_Out'
               )
compare.numeric('Mail_In',
               'Mail_In',
                label='Mail_In'
               )
compare.numeric('Mail_Out',
               'Mail_Out',
                label='Mail_Out'
               )

#Compute feature match score for each comparison factor for all the candidate matches
features = compare.compute(candidates, df_monthly_stats, df_monthly_stats).reset_index()

#Get a total score combining all feature scores
features['Score'] = features.loc[:, 'Port_Country_Score':'Mail_Out'].sum(axis=1)

#Merge dataframe back to original to match the old/new airline indexes to their respective names
result = pd.merge(df_last_occurrences[['Airline','datetime','Port_Country']], features, left_index=True,right_on=['level_0'],suffixes=('_old', ''))
result = pd.merge(df_first_occurrences[['Airline','datetime','Port_Country']], result, left_index=True,right_on=['level_1'],suffixes=('_new', '')).drop(columns=['level_0', 'level_1'])
result=result.rename(columns={'Airline':'Airline_old','datetime': 'datetime_old','Port_Country':'Port_Country_old'})

#Remove duplicate links, by keeping only the highest score for each old airline/new airline name combinations
result=result.sort_values('Score',ascending=False).drop_duplicates(subset=['Airline_old', 'Airline_new'], keep='first').reset_index(drop=True)

#As Port Countries stands out as one of the better indicators of airline similarity, assign it a higher weighting and recompute total score
result['Port_Country_Score']=result['Port_Country_Score']*3
result['Score'] = result.loc[:, 'Port_Country_Score':'Mail_Out'].sum(axis=1)
result.sort_values('Score',ascending=False).reset_index(drop=True)
#The result shows some strong links between old and new airline names. Google search suggests that most airlines in the top 15 matches
#have indeed gone through either name change or were part of a merger (notable exceptions are Kiwi Travel and Air France whose direct competitors started operating to/from Australia a month after they stopped)

,Airline_new,datetime_new,Port_Country_new,Airline_old,datetime_old,Port_Country_old,Port_Country_Score,Airline_Score,Passenger_In,Passenger_Out,Freight_In,Freight_Out,Mail_In,Mail_Out,Score
0,TIGERAIR,2011-01-01,Singapore,TIGER AIRWAYS,2010-12-01,Singapore,3.0,1.0,0.0,0.0,1.0000,1.000,1.0000,1.0000,8.0000
1,NAURU AIRLINES,2013-01-01,Nauru,OUR AIRLINE,2012-12-01,Nauru,3.0,1.0,0.0,0.0,0.9355,0.097,0.9965,0.9900,7.0190
2,FREEDOM AIR INTERNATIONAL,1996-10-01,New Zealand,KIWI TRAVEL INTL AIRLINES,1996-09-01,New Zealand,3.0,0.0,0.0,0.0,1.0000,1.000,1.0000,1.0000,7.0000
3,TASMAN CARGO AIRLINES,2008-01-01,New Zealand,ASIAN EXPRESS AIRLINES,2007-12-01,New Zealand,3.0,0.0,1.0,1.0,0.0000,0.000,1.0000,1.0000,7.0000
4,AUSTRIAN AIRLINES,2005-07-01,"Austria,Malaysia,Singapore",LAUDA AIR,2005-06-01,"Austria,Malaysia,Singapore",3.0,0.0,0.0,0.0,1.0000,1.000,1.0000,1.0000,7.0000
5,OUR AIRLINE,2008-01-01,"Kiribati,Nauru,Solomon Islands",AIR NAURU,2007-12-01,"Nauru,Solomon Islands",3.0,0.0,0.0,0.0,0.8705,0.836,0.9770,0.9535,6.6370
6,AIR CALEDONIE INTERNATIONAL,2013-01-01,New Caledonia,AIR CALEDONIE,2012-12-01,New Caledonia,3.0,1.0,0.0,0.0,0.7860,0.000,0.9785,0.6830,6.4475
7,LATAM AIRLINES,2016-06-01,"Chile,New Zealand",LAN AIRLINES,2016-05-01,"Chile,New Zealand",3.0,1.0,0.0,0.0,0.0000,0.000,0.9680,0.7125,5.6805
8,AOM FRENCH AIRLINES,1995-11-01,"France,New Caledonia,Sri Lanka",AIR FRANCE,1995-10-01,"France,Indonesia,New Caledonia,Singapore",3.0,0.0,0.0,0.0,0.5350,0.000,0.8385,0.8710,5.2445
9,LAN AIRLINES,2008-01-01,"Chile,New Zealand",LAN CHILE,2007-12-01,"Chile,New Zealand",3.0,1.0,0.0,0.0,0.0000,0.000,0.9400,0.0000,4.9400
